# AutoGen RoundRobinGroupChat
A team is a group of agents that work together to achieve a common goal.


## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [3]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)
# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team
Call the run() method to start the team with a task.

The termination condition was met when the word “APPROVE” is detected in the agent’s response. When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

In [4]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 49, 59, 378389, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=126), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 50, 2, 62631, tzinfo=datetime.timezone.utc), content="In autumn's embrace, the leaves ignite,  \nA tapestry of amber, gold, and bright.  \nWhispers of wind through the branches sway,  \nAs twilight descends, marking shorter days.  \n\nPumpkins abound in fields aglow,  \nWhile harvest moon casts a gentle show.  \nCrisp air carries scents of woodsmoke and spice,  \nNature's farewell, so tenderly nice.  \n\nThe world transforms in a vibrant parade,  \nWhere memories linger, and colors invade.  \nIn the fall's soft dance, we find our grace,  \nAs seasons shift, and time leaves its trace.  ", type='TextMe

## Observing a Team
Similar to the agent’s on_messages_stream() method, you can stream the team’s messages while it is running by calling the run_stream() method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the TaskResult object.

In [5]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 22, 20, 50, 51, 831588, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=146) metadata={} created_at=datetime.datetime(2025, 6, 22, 20, 50, 54, 161935, tzinfo=datetime.timezone.utc) content="In crisp embrace, the autumn calls,  \nWith golden leaves in fluttered falls,  \nA tapestry of red and gold,  \nSuch fleeting beauty to behold.  \n\nThe air is laced with pumpkin spice,  \nAs daylight dims, and shadows slice,  \nWith each rustle, whispers grow,  \nA symphony of sunlit glow.  \n\nThe harvest moon rides high and bright,  \nWhile chilly winds embrace the night,  \nIn every corner, nature sings,  \nA dance of change as summer swings.  \n\nSo gather 'round, let stories weave,  \nIn fall's embrace, we learn to cleave,  \nTo moments sweet, both small and grand,  \nIn autumn's arm

## Console()
The Console() method provides a convenient way to print messages to the console with proper formatting.

In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
In autumn's glow, the world transforms,  
As leaves like flames dance and swarm.  
Amber hues in crisp cool air,  
A tapestry woven with nature's care.  

Pumpkin patches line the fields,  
Harvest time, the bounty yields.  
Whispers of wind through trees so bare,  
A gentle reminder of summer's fare.  

Hot cocoa steams in twilight's embrace,  
While shadows stretch in a slower pace.  
As daylight fades and stars unfurl,  
The magic of fall, a cherished whirl.  
---------- TextMessage (critic) ----------
This is a lovely poem that beautifully captures the essence of the fall season. The imagery you use is vivid, and the emotional tone conveys both nostalgia and appreciation for nature's beauty during autumn. Here are a few suggestions for improvement:

1. **Imagery Expansion**: Consider adding more sensory details, such as sounds or smells, to further immerse 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 51, 22, 729555, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=110), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 51, 24, 449673, tzinfo=datetime.timezone.utc), content="In autumn's glow, the world transforms,  \nAs leaves like flames dance and swarm.  \nAmber hues in crisp cool air,  \nA tapestry woven with nature's care.  \n\nPumpkin patches line the fields,  \nHarvest time, the bounty yields.  \nWhispers of wind through trees so bare,  \nA gentle reminder of summer's fare.  \n\nHot cocoa steams in twilight's embrace,  \nWhile shadows stretch in a slower pace.  \nAs daylight fades and stars unfurl,  \nThe magic of fall, a cherished whirl.  ", type='TextMessage'), TextMessage(source='critic', models_

## Team has a State

In [7]:
await Console(team.run_stream(task="What was the last message?"))  # Stream the messages to the console.

---------- TextMessage (user) ----------
What was the last message?
---------- TextMessage (primary) ----------
The last message was a positive review of the revised poem about the fall season. It praised the incorporation of more sensory details that enriched the imagery and transported the reader into the experience of autumn. The critique noted the improved rhythm and flow of the poem and highlighted the strength of the closing lines in encapsulating the essence of fall and the sense of togetherness it fosters. Overall, it concluded with an enthusiastic "APPROVE" for the revisions made.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 51, 45, 557008, tzinfo=datetime.timezone.utc), content='What was the last message?', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=681, completion_tokens=91), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 51, 47, 302089, tzinfo=datetime.timezone.utc), content='The last message was a positive review of the revised poem about the fall season. It praised the incorporation of more sensory details that enriched the imagery and transported the reader into the experience of autumn. The critique noted the improved rhythm and flow of the poem and highlighted the strength of the closing lines in encapsulating the essence of fall and the sense of togetherness it fosters. Overall, it concluded with an enthusiastic "APPROVE" for the revisions made.', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

## Resetting a Team
You can reset the team by calling the reset() method. This method will clear the team’s state, including all agents. It will call the each agent’s on_reset() method

In [8]:
await team.reset()  # Reset the team for the next run.

## Stopping a Team
You can also stop the team from outside by using the ExternalTermination.

Calling set() on ExternalTermination will stop the team when the current agent’s turn is over. 

Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [9]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run # Primary agent is still allowed to complete its task.

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
In whispers soft, the autumn breeze,  
Dances through the rustling leaves,  
Gold and crimson, nature's art,  
As summer fades, it leaves its heart.  

The sun dips low, a golden hue,  
Painting skies in shades anew,  
Harvest moons and fields of grain,  
Time to gather, time to gain.  

Sweaters donned and fires ignited,  
Warmth of gatherings, happiness invited,  
With every crunch beneath our feet,  
Fall's sweet embrace is crisp and neat.  

So let the seasons shift and sway,  
In fall's embrace, we'll find our way,  
For though the days grow short and cool,  
In autumn's charm, we find the jewel.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 52, 25, 709287, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=149), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 52, 27, 994213, tzinfo=datetime.timezone.utc), content="In whispers soft, the autumn breeze,  \nDances through the rustling leaves,  \nGold and crimson, nature's art,  \nAs summer fades, it leaves its heart.  \n\nThe sun dips low, a golden hue,  \nPainting skies in shades anew,  \nHarvest moons and fields of grain,  \nTime to gather, time to gain.  \n\nSweaters donned and fires ignited,  \nWarmth of gatherings, happiness invited,  \nWith every crunch beneath our feet,  \nFall's sweet embrace is crisp and neat.  \n\nSo let the seasons shift and sway,  \nIn fall's embrace, we'll find our way

## Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the run() or run_stream() method again without a new task. RoundRobinGroupChat will continue from the next agent in the round-robin order.

In this example, critic agent continued where primary agent left off

In [10]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- TextMessage (critic) ----------
This poem beautifully captures the essence of the fall season with vivid imagery and a warm, inviting tone. The use of sensory language, like "whispers soft" and "crunch beneath our feet," allows readers to feel immersed in the experience of autumn. 

A few constructive suggestions:
1. **Rhythm and Flow**: Consider slightly revising some lines for smoother rhythm. For instance, "As summer fades, it leaves its heart" could be adjusted to improve meter.
2. **Imagery Expansion**: While the imagery is strong, you might enhance it with additional elements of fall, such as specific animals or activities associated with the season, to deepen the sense of place.
3. **Ending Strength**: The closing lines encapsulate the theme well, but you could emphasize the "jewel" metaphor further, perhaps by elaborating on what this jewel represents—nostalgia, beauty, change, etc.

Overall, it's a lovely reflection on autumn, and with these minor adjustments, it co

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=194, completion_tokens=215), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 52, 44, 77004, tzinfo=datetime.timezone.utc), content='This poem beautifully captures the essence of the fall season with vivid imagery and a warm, inviting tone. The use of sensory language, like "whispers soft" and "crunch beneath our feet," allows readers to feel immersed in the experience of autumn. \n\nA few constructive suggestions:\n1. **Rhythm and Flow**: Consider slightly revising some lines for smoother rhythm. For instance, "As summer fades, it leaves its heart" could be adjusted to improve meter.\n2. **Imagery Expansion**: While the imagery is strong, you might enhance it with additional elements of fall, such as specific animals or activities associated with the season, to deepen the sense of place.\n3. **Ending Strength**: The closing lines encapsulate the theme well, but you could emphasize the "

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [11]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


## Other Built-In Termination Conditions:
- MaxMessageTermination: Stops after a specified number of messages have been produced, including both agent and task messages.
- TextMentionTermination: Stops when specific text or string is mentioned in a message (e.g., “TERMINATE”).
- ExternalTermination: Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., “Stop” buttons in chat interfaces).
- TokenUsageTermination: Stops when a certain number of prompt or completion tokens are used. This requires the agents to report token usage in their messages.
- TimeoutTermination: Stops after a specified duration in seconds.
- SourceMatchTermination: Stops after a specific agent responds.
- HandoffTermination: Stops when a handoff to a specific target is requested. Handoff messages can be used to build patterns such as Swarm. This is useful when you want to pause the run and allow application or user to provide input when an agent hands off to them.
- StopMessageTermination: Stops when a StopMessage is produced by an agent.

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination, TokenUsageTermination, TimeoutTermination, SourceMatchTermination, HandoffTermination, StopMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the first agent.
first_agent = AssistantAgent(
    "first",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
second_agent = AssistantAgent(
    "second",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# Create the second agent.
third_agent = AssistantAgent(
    "third",
    model_client=az_model_client,
    system_message="You will give a random number between 1 and 10.",
)

# MaxMessageTermination
termination = MaxMessageTermination(5) # terminate after 5 messages
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

await Console(team.run_stream(task="Give a number"))

---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
7
---------- TextMessage (second) ----------
5
---------- TextMessage (third) ----------
3
---------- TextMessage (first) ----------
2


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 53, 50, 185816, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 53, 51, 402913, tzinfo=datetime.timezone.utc), content='7', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 53, 51, 901500, tzinfo=datetime.timezone.utc), content='5', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 53, 52, 384207, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=47, comple

In [13]:
#TextMentionTermination
termination = TextMentionTermination("4") # Terminate when the message contains "4".
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
4


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 0, 640991, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 1, 771725, tzinfo=datetime.timezone.utc), content='4', type='TextMessage')], stop_reason="Text '4' mentioned")

In [14]:
import asyncio
#ExternalTermination
termination = ExternalTermination() # Create an external termination condition.

team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Give a number")))

# Wait for some time.
await asyncio.sleep(3)

# Stop the team.
termination.set()

# Wait for the team to finish.
await run


---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
9
---------- TextMessage (second) ----------
3
---------- TextMessage (third) ----------
6
---------- TextMessage (first) ----------
1
---------- TextMessage (second) ----------
8


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 22, 217484, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=73, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 23, 606460, tzinfo=datetime.timezone.utc), content='9', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=53, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 24, 103496, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 24, 589306, tzinfo=datetime.timezone.utc), content='6', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=92, comple

In [15]:
#TokenUsageTermination
termination = TokenUsageTermination(100) # terminate when token usage is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
8


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 53, 911713, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 54, 55, 161231, tzinfo=datetime.timezone.utc), content='8', type='TextMessage')], stop_reason='Token usage limit reached, total token count: 107, prompt token count: 105, completion token count: 2.')

In [16]:
#TimeoutTermination
termination = TimeoutTermination(5) # terminate when time out is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
10
---------- TextMessage (second) ----------
2
---------- TextMessage (third) ----------
4
---------- TextMessage (first) ----------
5
---------- TextMessage (second) ----------
9
---------- TextMessage (third) ----------
1
---------- TextMessage (first) ----------
6
---------- TextMessage (second) ----------
7
---------- TextMessage (third) ----------
2


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 3, 981706, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 5, 215065, tzinfo=datetime.timezone.utc), content='10', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=90, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 5, 716384, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=93, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 6, 215797, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=137, complet

In [17]:
#SourceMatchTermination
termination = SourceMatchTermination("third") # terminate when source match agent is met.
team = RoundRobinGroupChat([first_agent, second_agent, third_agent], termination_condition=termination)
await Console(team.run_stream(task="Give a number"))

---------- TextMessage (user) ----------
Give a number
---------- TextMessage (first) ----------
3
---------- TextMessage (second) ----------
4
---------- TextMessage (third) ----------
8


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 17, 121357, tzinfo=datetime.timezone.utc), content='Give a number', type='TextMessage'), TextMessage(source='first', models_usage=RequestUsage(prompt_tokens=169, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 18, 305344, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(source='second', models_usage=RequestUsage(prompt_tokens=145, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 18, 789323, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(source='third', models_usage=RequestUsage(prompt_tokens=155, completion_tokens=2), metadata={}, created_at=datetime.datetime(2025, 6, 22, 20, 55, 19, 278943, tzinfo=datetime.timezone.utc), content='8', type='TextMessage')], stop_reason="'third' answered")